In [1]:
import pandas as pd
import numpy as np
import psycopg2

In [2]:
# Primero tenemos que conectarnos a la base de datos
conn = psycopg2.connect(
    dbname="ITunes",
    user="postgres",
    password="admin",
    host="localhost",
    port="5432"
)

In [3]:
# Creamos un cursor que nos permita ejecutar consultas a esta conexión que acabamos de hacer
cur = conn.cursor()

In [4]:
# Comprobamos la conexión
cur.execute("SELECT version();")
print(cur.fetchone())

('PostgreSQL 16.3, compiled by Visual C++ build 1939, 64-bit',)


In [5]:
# Cargamos los archivos necesarios para la tabla genre
df_genre = pd.read_pickle("../data/data_limpio/genre.pkl")
df_genre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290 entries, 0 to 289
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   genre_id          290 non-null    int64 
 1   primarygenrename  290 non-null    object
dtypes: int64(1), object(1)
memory usage: 4.7+ KB


In [6]:
tabla_genre = df_genre
tabla_genre.head()

,genre_id,primarygenrename
0,0,Blues
1,1,Soundtrack
2,2,Rock
3,3,Spoken Word
4,4,R&BSoul


In [7]:
# Creamos la consulta
insert_query = """
    INSERT INTO genre (genre_id, primarygenrename)
    VALUES (%s, %s)
"""

In [ ]:
# Itera sobre el DataFrame df_genre y extrae los valores de cada fila
# Para construir una lista de listas llamada data_to_insert
# Esta lista puede usarse después para inserciones en bloque (por ejemplo, con executemany).

data_to_insert = []
for _, row in df_genre.iterrows():
    genre_id = row["genre_id"]
    primarygenrename = row["primarygenrename"]
    
    data_to_insert.append([genre_id, primarygenrename])

In [9]:
# Insertamos usando el executemany()
cur.executemany(insert_query, data_to_insert)
# Guardamos los cambios
conn.commit()

In [10]:
# Comprobamos en la base de datos. Podemos hacerlo desde python
query_comprobacion = """
                SELECT * FROM genre LIMIT 10
"""
cur.execute(query_comprobacion)
cur.fetchall()

[(0, 'Blues'),
 (1, 'Soundtrack'),
 (2, 'Rock'),
 (3, 'Spoken Word'),
 (4, 'R&BSoul'),
 (5, 'Pop'),
 (6, 'Dance'),
 (7, 'SingerSongwriter'),
 (8, 'Hip-Hop'),
 (9, 'Worldwide')]

In [11]:
# Cargamos los archivos necesarios para la tabla artist
df_artist = pd.read_pickle("../data/data_limpio/artist.pkl")
df_artist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22460 entries, 0 to 22459
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   artist_id      22460 non-null  int64 
 1   artistname     22460 non-null  object
 2   artistviewurl  22460 non-null  object
dtypes: int64(1), object(2)
memory usage: 526.5+ KB


In [12]:
tabla_artist = df_artist
tabla_artist.head()

,artist_id,artistname,artistviewurl
0,10323,Leo Kottke,httpsmusic.apple.comusartistleo-kottke10323?uo4
1,10416,Djavan,httpsmusic.apple.comusartistdjavan10416?uo4
2,10735,Taj Mahal & Ry Cooder,httpsmusic.apple.comusartisttaj-mahal10735?uo4
3,10811,Etta James,httpsmusic.apple.comusartistetta-james10811?uo4
4,11175,Peabo Bryson,httpsmusic.apple.comusartistpeabo-bryson11175?uo4


In [17]:
# Creamos la consulta
insert_query = """
    INSERT INTO artist (artist_id, artistname, artistviewurl)
    VALUES (%s, %s, %s)
"""

In [18]:
# Itera sobre el DataFrame df_artist y extrae los valores de cada fila
# Para construir una lista de listas llamada data_to_insert
# Esta lista puede usarse después para inserciones en bloque (por ejemplo, con executemany).

data_to_insert = []
for _, row in df_artist.iterrows():
    artist_id = row["artist_id"]
    artistname = row["artistname"]
    artistviewurl = row["artistviewurl"]
    
    data_to_insert.append([artist_id, artistname, artistviewurl])

In [20]:
conn.rollback()

In [21]:
# Insertamos usando el executemany()
cur.executemany(insert_query, data_to_insert)
# Guardamos los cambios
conn.commit()

In [22]:
# Comprobamos en la base de datos. Podemos hacerlo desde python
query_comprobacion = """
                SELECT * FROM artist LIMIT 10
"""
cur.execute(query_comprobacion)
cur.fetchall()

[(10323, 'Leo Kottke', 'httpsmusic.apple.comusartistleo-kottke10323?uo4'),
 (10416, 'Djavan', 'httpsmusic.apple.comusartistdjavan10416?uo4'),
 (10735,
  'Taj Mahal & Ry Cooder',
  'httpsmusic.apple.comusartisttaj-mahal10735?uo4'),
 (10811, 'Etta James', 'httpsmusic.apple.comusartistetta-james10811?uo4'),
 (11175, 'Peabo Bryson', 'httpsmusic.apple.comusartistpeabo-bryson11175?uo4'),
 (11697,
  'Nightnoise & Triona Ni Dhomhnaill',
  'httpsmusic.apple.comusartistnightnoise11697?uo4'),
 (12043,
  'Torcuato Mariano',
  'httpsmusic.apple.comusartisttorcuato-mariano12043?uo4'),
 (12113,
  'Babyface, Mariah Carey, Kenny G & Sheila E.',
  'httpsmusic.apple.comusartistbabyface12113?uo4'),
 (12224,
  "Paul McCartney, Bon Jovi, Sheryl Crow, Fergie, Colbie Caillat, Natasha Bedingfield, Leona Lewis, Sean Paul, John Rzeznik, Krewella, Angelique Kidjo, Kelsea Ballerini, Nicole Scherzinger, Christina Grimmie, Victoria Justice & Q'orianka Kilcher",
  'httpsmusic.apple.comusartistpaul-mccartney12224?uo4'

In [23]:
# Cargamos los archivos necesarios para la tabla album
df_album = pd.read_pickle("../data/data_limpio/album.pkl")
df_album.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51056 entries, 0 to 118807
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   collection_id            51056 non-null  Int64         
 1   collectionname           51056 non-null  object        
 2   collectioncensoredname   51056 non-null  object        
 3   release_date             51056 non-null  datetime64[ns]
 4   collectionexplicitness   51056 non-null  object        
 5   contentadvisoryrating    51056 non-null  object        
 6   collectionprice          51056 non-null  float64       
 7   currency                 51056 non-null  object        
 8   trackcount               51056 non-null  Int64         
 9   disccount                51056 non-null  Int64         
 10  collectionviewurl        51056 non-null  object        
 11  collectionartist_id      51056 non-null  Int64         
 12  collectionartistname     51056 non-n

In [24]:
tabla_album = df_album
tabla_album.head()

,collection_id,collectionname,collectioncensoredname,release_date,collectionexplicitness,contentadvisoryrating,collectionprice,currency,trackcount,disccount,collectionviewurl,collectionartist_id,collectionartistname,collectionartistviewurl,artist_id
0,40459605,Briefcase Full of Blues,Briefcase Full of Blues,1978-11-01,notExplicit,Sin identificar,9.99,USD,12,1,httpsmusic.apple.comusalbumsoul-man40459605?i4...,1,Sin identificar,Sin identificar,14406835
1,452584443,The Blues Brothers Original Soundtrack Recording,The Blues Brothers Original Soundtrack Recording,1990-01-01,notExplicit,Sin identificar,10.99,USD,11,1,httpsmusic.apple.comusalbumeverybody-needs-som...,2,Sin identificar,Sin identificar,14406835
19,1203656134,Blues Gems,Blues Gems,1980-06-01,notExplicit,Sin identificar,9.99,USD,22,1,httpsmusic.apple.comusalbumsweet-home-chicago1...,20,Various Artists,Sin identificar,14406835
20,797529975,Made In America,Made In America,1980-12-05,notExplicit,Sin identificar,9.99,USD,10,1,httpsmusic.apple.comusalbumwhos-making-love797...,21,Sin identificar,Sin identificar,14406835
30,321029302,Soul Man Excusez Moi Mon Cherie Digital 45 - S...,Soul Man Excusez Moi Mon Cherie Digital 45 - S...,1978-01-01,notExplicit,Sin identificar,1.99,USD,2,1,httpsmusic.apple.comusalbumsoul-man321029302?i...,31,Sin identificar,Sin identificar,14406835


In [25]:
# Creamos la consulta
insert_query = """
    INSERT INTO album (collection_id, collectionname, collectioncensoredname, release_date, collectionexplicitness,
      contentadvisoryrating, collectionprice, currency, trackcount, disccount, collectionviewurl,
        collectionartistname, collectionartistviewurl, artist_id)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

In [ ]:
# Itera sobre el DataFrame df_artist y extrae los valores de cada fila
# Para construir una lista de listas llamada data_to_insert
# Esta lista puede usarse después para inserciones en bloque (por ejemplo, con executemany).
 
data_to_insert = []
for _, row in df_album.iterrows():
    collection_id = row["collection_id"]
    collectionname = row["collectionname"]
    collectioncensoredname = row["collectioncensoredname"]
    release_date = row["release_date"]
    collectionexplicitness = row["collectionexplicitness"]
    contentadvisoryrating = row["contentadvisoryrating"]
    collectionprice = row["collectionprice"]
    currency = row["currency"]
    trackcount = row["trackcount"]
    disccount = row["disccount"]
    collectionviewurl = row["collectionviewurl"]
    collectionartistname = row["collectionartistname"]
    collectionartistviewurl = row["collectionartistviewurl"]
    artist_id = row["artist_id"]
    
    data_to_insert.append([collection_id, collectionname, collectioncensoredname, release_date, collectionexplicitness,
                          contentadvisoryrating, collectionprice, currency, trackcount, disccount,
                           collectionviewurl, collectionartistname, collectionartistviewurl, artist_id])

In [27]:
# Insertamos usando el executemany()
cur.executemany(insert_query, data_to_insert)
# Guardamos los cambios
conn.commit()

In [28]:
# Comprobamos en la base de datos. Podemos hacerlo desde python
query_comprobacion = """
                SELECT * FROM album LIMIT 10
"""
cur.execute(query_comprobacion)
cur.fetchall()

[(40459605,
  'Briefcase Full of Blues',
  'Briefcase Full of Blues',
  datetime.date(1978, 11, 1),
  'notExplicit',
  'Sin identificar',
  9.99,
  'USD',
  12,
  1,
  'httpsmusic.apple.comusalbumsoul-man40459605?i40459637&uo4',
  'Sin identificar',
  'Sin identificar',
  14406835),
 (452584443,
  'The Blues Brothers Original Soundtrack Recording',
  'The Blues Brothers Original Soundtrack Recording',
  datetime.date(1990, 1, 1),
  'notExplicit',
  'Sin identificar',
  10.99,
  'USD',
  11,
  1,
  'httpsmusic.apple.comusalbumeverybody-needs-somebody-to-love452584443?i452584459&uo4',
  'Sin identificar',
  'Sin identificar',
  14406835),
 (1203656134,
  'Blues Gems',
  'Blues Gems',
  datetime.date(1980, 6, 1),
  'notExplicit',
  'Sin identificar',
  9.99,
  'USD',
  22,
  1,
  'httpsmusic.apple.comusalbumsweet-home-chicago1203656134?i1203656841&uo4',
  'Various Artists',
  'Sin identificar',
  14406835),
 (797529975,
  'Made In America',
  'Made In America',
  datetime.date(1980, 12, 5

In [29]:
# Cargamos los archivos necesarios para la tabla track
df_track = pd.read_pickle("../data/data_limpio/track.pkl")
df_track.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110567 entries, 0 to 110566
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   track_id           110567 non-null  int64         
 1   trackname          110567 non-null  object        
 2   tracknumber        110567 non-null  Int64         
 3   trackprice         110567 non-null  float64       
 4   discnumber         110567 non-null  Int64         
 5   tracktimemillis    110567 non-null  Int64         
 6   trackexplicitness  110567 non-null  object        
 7   release_date       110567 non-null  datetime64[ns]
 8   trackviewurl       110567 non-null  object        
 9   is_streamable      110567 non-null  boolean       
 10  kind               110567 non-null  object        
 11  artist_id          110567 non-null  int64         
 12  collection_id      110567 non-null  Int64         
 13  genre_id           110567 non-null  int64   

In [30]:
tabla_track = df_track
tabla_track.head()

,track_id,trackname,tracknumber,trackprice,discnumber,tracktimemillis,trackexplicitness,release_date,trackviewurl,is_streamable,kind,artist_id,collection_id,genre_id
0,40459637,Soul Man,9,1.29,1,208498,notExplicit,1978-11-01,httpsmusic.apple.comusalbumsoul-man40459605?i4...,True,song,14406835,40459605,0
1,452584459,Everybody Needs Somebody to Love,5,1.29,1,200733,notExplicit,1990-01-01,httpsmusic.apple.comusalbumeverybody-needs-som...,True,song,14406835,452584443,1
2,452584447,Peter Gunn Theme,2,1.29,1,230333,notExplicit,1980-06-01,httpsmusic.apple.comusalbumpeter-gunn-theme452...,True,song,14406835,452584443,1
3,452584465,Theme from Rawhide,8,1.29,1,159600,notExplicit,1980-06-01,httpsmusic.apple.comusalbumtheme-from-rawhide4...,True,song,14406835,452584443,1
4,40459628,Rubber Biscuit,5,1.29,1,177800,notExplicit,1978-11-01,httpsmusic.apple.comusalbumrubber-biscuit40459...,True,song,14406835,40459605,0


In [31]:
# Creamos la consulta
insert_query = """
    INSERT INTO track (track_id, trackname, tracknumber, trackprice, discnumber,
      tracktimemillis, trackexplicitness, release_date, trackviewurl, is_streamable, kind,
        artist_id, collection_id, genre_id)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

In [ ]:
# Itera sobre el DataFrame df_track y extrae los valores de cada fila
# Para construir una lista de listas llamada data_to_insert
# Esta lista puede usarse después para inserciones en bloque (por ejemplo, con executemany).

data_to_insert = []
for _, row in df_track.iterrows():
    track_id = row["track_id"]
    trackname = row["trackname"]
    tracknumber = row["tracknumber"]
    trackprice = row["trackprice"]
    discnumber = row["discnumber"]
    tracktimemillis = row["tracktimemillis"]
    trackexplicitness = row["trackexplicitness"]
    release_date = row["release_date"]
    trackviewurl = row["trackviewurl"]
    is_streamable = row["is_streamable"]
    kind = row["kind"]
    artist_id = row["artist_id"]
    collection_id = row["collection_id"]
    genre_id = row["genre_id"]
    
    data_to_insert.append([track_id, trackname, tracknumber, trackprice, discnumber,
                            tracktimemillis, trackexplicitness, release_date, trackviewurl, is_streamable,
                          kind, artist_id, collection_id, genre_id])

In [33]:
# Insertamos usando el executemany()
cur.executemany(insert_query, data_to_insert)
# Guardamos los cambios
conn.commit()

In [34]:
# Comprobamos en la base de datos. Podemos hacerlo desde python
query_comprobacion = """
                SELECT * FROM track LIMIT 10
"""
cur.execute(query_comprobacion)
cur.fetchall()

[(40459637,
  'Soul Man',
  9,
  1.29,
  1,
  208498,
  'notExplicit',
  datetime.date(1978, 11, 1),
  'httpsmusic.apple.comusalbumsoul-man40459605?i40459637&uo4',
  True,
  'song',
  14406835,
  40459605,
  0),
 (452584459,
  'Everybody Needs Somebody to Love',
  5,
  1.29,
  1,
  200733,
  'notExplicit',
  datetime.date(1990, 1, 1),
  'httpsmusic.apple.comusalbumeverybody-needs-somebody-to-love452584443?i452584459&uo4',
  True,
  'song',
  14406835,
  452584443,
  1),
 (452584447,
  'Peter Gunn Theme',
  2,
  1.29,
  1,
  230333,
  'notExplicit',
  datetime.date(1980, 6, 1),
  'httpsmusic.apple.comusalbumpeter-gunn-theme452584443?i452584447&uo4',
  True,
  'song',
  14406835,
  452584443,
  1),
 (452584465,
  'Theme from Rawhide',
  8,
  1.29,
  1,
  159600,
  'notExplicit',
  datetime.date(1980, 6, 1),
  'httpsmusic.apple.comusalbumtheme-from-rawhide452584443?i452584465&uo4',
  True,
  'song',
  14406835,
  452584443,
  1),
 (40459628,
  'Rubber Biscuit',
  5,
  1.29,
  1,
  177800,

In [35]:
# Cargamos los archivos necesarios para la tabla album_prices
df_album_prices = pd.read_pickle("../data/data_limpio/album_prices.pkl")
df_album_prices.info()

<class 'pandas.core.frame.DataFrame'>
Index: 54219 entries, 0 to 118807
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   collection_id    54219 non-null  Int64         
 1   collectionprice  54219 non-null  float64       
 2   checked_at       54219 non-null  datetime64[ns]
dtypes: Int64(1), datetime64[ns](1), float64(1)
memory usage: 1.7 MB


In [36]:
tabla_album_prices = df_album_prices
tabla_album_prices.head()

,collection_id,collectionprice,checked_at
0,40459605,9.99,2025-04-20
1,452584443,10.99,2025-04-20
19,1203656134,9.99,2025-04-20
20,797529975,9.99,2025-04-20
30,321029302,1.99,2025-04-20


In [37]:
# Creamos la consulta
insert_query = """
    INSERT INTO album_prices (collection_id, collectionprice, checked_at)
    VALUES (%s, %s, %s)
"""

In [40]:
# Itera sobre el DataFrame df_album_prices y extrae los valores de cada fila
# Para construir una lista de listas llamada data_to_insert
# Esta lista puede usarse después para inserciones en bloque (por ejemplo, con executemany).

data_to_insert = []
for _, row in df_album_prices.iterrows():
    collection_id = row["collection_id"]
    collectionprice = row["collectionprice"]
    checked_at = row["checked_at"]
    
    data_to_insert.append([collection_id, collectionprice, checked_at])

In [41]:
# Insertamos usando el executemany()
cur.executemany(insert_query, data_to_insert)
# Guardamos los cambios
conn.commit()

In [42]:
# Comprobamos en la base de datos. Podemos hacerlo desde python
query_comprobacion = """
                SELECT * FROM album_prices LIMIT 10
"""
cur.execute(query_comprobacion)
cur.fetchall()

[(1, 9.99, datetime.date(2025, 4, 20), 40459605),
 (2, 10.99, datetime.date(2025, 4, 20), 452584443),
 (3, 9.99, datetime.date(2025, 4, 20), 1203656134),
 (4, 9.99, datetime.date(2025, 4, 20), 797529975),
 (5, 1.99, datetime.date(2025, 4, 20), 321029302),
 (6, 10.99, datetime.date(2025, 4, 20), 1237659225),
 (7, 9.99, datetime.date(2025, 4, 20), 1440722180),
 (8, 1.49, datetime.date(2025, 4, 20), 321026122),
 (9, 9.99, datetime.date(2025, 4, 20), 274696705),
 (10, 10.99, datetime.date(2025, 4, 20), 1213306014)]

In [43]:
# Cargamos los archivos necesarios para la tabla track_prices
df_track_prices = pd.read_pickle("../data/data_limpio/track_prices.pkl")
df_track_prices.info()

<class 'pandas.core.frame.DataFrame'>
Index: 116217 entries, 0 to 118807
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   track_id    116217 non-null  int64         
 1   trackprice  116217 non-null  float64       
 2   checked_at  116217 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 3.5 MB


In [44]:
tabla_track_prices = df_track_prices
tabla_track_prices.head()

,track_id,trackprice,checked_at
0,40459637,1.29,2025-04-20
1,452584459,1.29,2025-04-20
2,452584447,1.29,2025-04-20
3,452584465,1.29,2025-04-20
4,40459628,1.29,2025-04-20


In [45]:
# Creamos la consulta
insert_query = """
    INSERT INTO track_prices (track_id, trackprice, checked_at)
    VALUES (%s, %s, %s)
"""

In [46]:
# Itera sobre el DataFrame df_track_prices y extrae los valores de cada fila
# Para construir una lista de listas llamada data_to_insert
# Esta lista puede usarse después para inserciones en bloque (por ejemplo, con executemany).

data_to_insert = []
for _, row in df_track_prices.iterrows():
    track_id = row["track_id"]
    trackprice = row["trackprice"]
    checked_at = row["checked_at"]
    
    data_to_insert.append([track_id, trackprice, checked_at])

In [47]:
# Insertamos usando el executemany()
cur.executemany(insert_query, data_to_insert)
# Guardamos los cambios
conn.commit()

In [48]:
# Comprobamos en la base de datos. Podemos hacerlo desde python
query_comprobacion = """
                SELECT * FROM track_prices LIMIT 10
"""
cur.execute(query_comprobacion)
cur.fetchall()

[(1, 1.29, datetime.date(2025, 4, 20), 40459637),
 (2, 1.29, datetime.date(2025, 4, 20), 452584459),
 (3, 1.29, datetime.date(2025, 4, 20), 452584447),
 (4, 1.29, datetime.date(2025, 4, 20), 452584465),
 (5, 1.29, datetime.date(2025, 4, 20), 40459628),
 (6, 1.29, datetime.date(2025, 4, 20), 452584448),
 (7, 1.12, datetime.date(2025, 4, 20), 452584468),
 (8, 1.12, datetime.date(2025, 4, 20), 452584467),
 (9, 1.29, datetime.date(2025, 4, 20), 452584446),
 (10, 1.29, datetime.date(2025, 4, 20), 40459620)]